# Find the position of the information

In [3]:
import requests
from bs4 import BeautifulSoup

URL = "https://m.bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

# class="roundy" is the table contain all pokemon

# Each row start with (<tr style="background:#FFF">) end with </tr> contain the "Ndex","MS","Pokémon","Type"

# Example of the link to the information wensite, <a href="/wiki/Ivysaur_(Pok%C3%A9mon)" title="Ivysaur (Pokémon)">Ivysaur</a>
# Title is the name of the pokemon, the href = the link to the information website

<!DOCTYPE html>

<html class="client-nojs mf-expand-sections-clientpref-0 mf-font-size-clientpref-small mw-mf-amc-clientpref-0" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Pokémon by National Pokédex number - Bulbapedia, the community-driven Pokémon encyclopedia</title>
<script>(function(){var className="client-js mf-expand-sections-clientpref-0 mf-font-size-clientpref-small mw-mf-amc-clientpref-0";var cookie=document.cookie.match(/(?:^|; )themozz_mediawikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split('%2C').forEach(function(pref){className=className.replace(new RegExp('(^| )'+pref.replace(/-clientpref-\w+$|[^\w-]+/g,'')+'-clientpref-\\w+( |$)'),'$1'+pref+'$2');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November

In [5]:
import requests
from bs4 import BeautifulSoup

URL = "https://m.bulbapedia.bulbagarden.net/wiki/Bulbasaur_(Pok%C3%A9mon)"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

# Class "roundy infobox" is the information box that contain types, height, weight, etc.
# <table class="roundy infobox" style="float:right; text-align:center; width:33%; max-width:420px; background: #3FA129; border: 2px solid #5E2A84; padding:2px;">

# Section class "mf-section-1 collapsible-block collapsible-block-js open-block" contain bio
# <section class="mf-section-1 collapsible-block collapsible-block-js open-block" id="content-collapsible-block-0">

<!DOCTYPE html>

<html class="client-nojs mf-expand-sections-clientpref-0 mf-font-size-clientpref-small mw-mf-amc-clientpref-0" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Bulbasaur (Pokémon) - Bulbapedia, the community-driven Pokémon encyclopedia</title>
<script>(function(){var className="client-js mf-expand-sections-clientpref-0 mf-font-size-clientpref-small mw-mf-amc-clientpref-0";var cookie=document.cookie.match(/(?:^|; )themozz_mediawikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split('%2C').forEach(function(pref){className=className.replace(new RegExp('(^| )'+pref.replace(/-clientpref-\w+$|[^\w-]+/g,'')+'-clientpref-\\w+( |$)'),'$1'+pref+'$2');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgReques

# Extract information

In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL for constructing full detail page URLs.
base_url = "https://m.bulbapedia.bulbagarden.net"

# URL for the listing page that contains Pokémon grouped by generation.
listing_url = "https://m.bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

# Request the listing page.
response = requests.get(listing_url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch {listing_url}")

# Parse the listing page.
soup = BeautifulSoup(response.content, "html.parser")

# Find all rows that appear to represent Pokémon entries.
# The Pokémon rows have a style attribute with "background:#FFF"
rows = soup.find_all("tr", style=lambda value: value and "background:#FFF" in value)

data = []
print(f"Found {len(rows)} Pokémon rows across all generations.")

for row in rows:
    # Get all cell elements from the row.
    cells = row.find_all("td")
    # Skip rows that don't have at least three cells (dex number, image, name)
    if len(cells) < 3:
        continue

    # Extract Name and Link
    # The 3rd cell (index 2) should contain the link with the Pokémon's name.
    name_cell = cells[2]
    # Look for an anchor whose title contains "(Pokémon)".
    name_link = name_cell.find("a", title=lambda t: t and "(Pokémon)" in t)
    if not name_link:
        continue
    # Optionally, clean up the title (for example, removing " (Pokémon)" if desired)
    name = name_link.get("title").replace(" (Pokémon)", "")
    rel_link = name_link.get("href")
    if not rel_link:
        continue

    # Extract Type(s)
    # Look for all links in the row with title containing "(type)"
    type_tags = row.find_all("a", title=lambda x: x and "(type)" in x)
    # Create a list of type names (e.g., "Grass", "Poison")
    types = [tag.get_text(strip=True) for tag in type_tags]
    # Optionally, you could join them into a single comma separated string:
    types_str = ", ".join(types) if types else None

    # Construct the full URL for the Pokémon detail page.
    if rel_link.startswith("/"):
        pokemon_url = base_url + rel_link
    else:
        pokemon_url = rel_link

    print(f"Processing {name} ({types_str}): {pokemon_url}")
    
    # Get the Detail Page 
    detail_response = requests.get(pokemon_url)
    if detail_response.status_code != 200:
        print(f"Warning: Could not fetch {pokemon_url} (status code {detail_response.status_code})")
        continue

    detail_soup = BeautifulSoup(detail_response.content, "html.parser")

    # Extract Information Table
    info_table = detail_soup.find("table", class_="roundy infobox")
    info_html = info_table.prettify() if info_table else None

    # Extract Bio
    # Use the previously confirmed CSS selector.
    bio_section = detail_soup.select_one("section.mf-section-1.collapsible-block")
    bio_text = bio_section.get_text(separator=" ", strip=True) if bio_section else None

    # Add all collected information into our data list.
    data.append({
        "name": name,
        "Type": types_str,
        "information table": info_html,
        "Bio": bio_text
    })

    # Pause briefly to avoid hammering the server.
    time.sleep(1)

# Create a DataFrame from the collected data.
df = pd.DataFrame(data)
print(df.head())

Found 1181 Pokémon rows across all generations.
Processing Bulbasaur (Grass, Poison): https://m.bulbapedia.bulbagarden.net/wiki/Bulbasaur_(Pok%C3%A9mon)
Processing Ivysaur (Grass, Poison): https://m.bulbapedia.bulbagarden.net/wiki/Ivysaur_(Pok%C3%A9mon)
Processing Venusaur (Grass, Poison): https://m.bulbapedia.bulbagarden.net/wiki/Venusaur_(Pok%C3%A9mon)
Processing Charmander (Fire): https://m.bulbapedia.bulbagarden.net/wiki/Charmander_(Pok%C3%A9mon)
Processing Charmeleon (Fire): https://m.bulbapedia.bulbagarden.net/wiki/Charmeleon_(Pok%C3%A9mon)
Processing Charizard (Fire, Flying): https://m.bulbapedia.bulbagarden.net/wiki/Charizard_(Pok%C3%A9mon)
Processing Squirtle (Water): https://m.bulbapedia.bulbagarden.net/wiki/Squirtle_(Pok%C3%A9mon)
Processing Wartortle (Water): https://m.bulbapedia.bulbagarden.net/wiki/Wartortle_(Pok%C3%A9mon)
Processing Blastoise (Water): https://m.bulbapedia.bulbagarden.net/wiki/Blastoise_(Pok%C3%A9mon)
Processing Caterpie (Bug): https://m.bulbapedia.bulbag

In [57]:
df

,name,Type,information table,Bio
0,Bulbasaur,"Grass, Poison","<table class=""roundy infobox"" style=""float:rig...",Bulbasaur using its vines to climb a tree Bulb...
1,Ivysaur,"Grass, Poison","<table class=""roundy infobox"" style=""float:rig...",Ivysaur is a quadrupedal amphibian Pokémon tha...
2,Venusaur,"Grass, Poison","<table class=""roundy infobox"" style=""float:rig...",Venusaur with its bulb closed Venusaur is a sq...
3,Charmander,Fire,"<table class=""roundy infobox"" style=""float:rig...",Charmander protecting the flame on its tail fr...
4,Charmeleon,Fire,"<table class=""roundy infobox"" style=""float:rig...","Charmeleon is a bipedal, reptilian Pokémon . I..."
...,...,...,...,...
1020,Raging Bolt,"Electric, Dragon","<table class=""roundy infobox"" style=""float:rig...",Raging Bolt's mane unfolding Raging Bolt is a ...
1021,Iron Boulder,"Rock, Psychic","<table class=""roundy infobox"" style=""float:rig...",Iron Boulder's head rotating as it faints Iron...
1022,Iron Crown,"Steel, Psychic","<table class=""roundy infobox"" style=""float:rig...",Iron Crown's blades extending while using Tach...
1023,Terapagos,Normal,"<table class=""roundy infobox"" style=""float:rig...",Terapagos in its dormant state Normal Form Ter...


In [59]:
import pandas as pd
from bs4 import BeautifulSoup

# Define a helper function that extracts the desired information from the HTML string
def extract_info(info_html):
    soup = BeautifulSoup(info_html, "html.parser")
    
    # Extract Pokémon Category
    category_tag = soup.find("a", title="Pokémon category")
    category = category_tag.get_text(strip=True) if category_tag else None

    # Extract Height
    height_imperial, height_metric = None, None
    height_tag = soup.find("a", title="List of Pokémon by height")
    if height_tag:
        height_td = height_tag.find_parent("td")
        if height_td:
            # Look for the inner table that contains the height values
            height_table = height_td.find("table")
            if height_table:
                first_tr = height_table.find("tr")
                if first_tr:
                    tds = first_tr.find_all("td")
                    if len(tds) >= 2:
                        height_imperial = tds[0].get_text(strip=True)
                        height_metric = tds[1].get_text(strip=True)
    # Combine the two values (or choose one)
    height = f"{height_imperial} ({height_metric})" if height_imperial and height_metric else height_imperial

    # Extract Weight
    weight_imperial, weight_metric = None, None
    weight_tag = soup.find("a", title="Weight")
    if weight_tag:
        weight_td = weight_tag.find_parent("td")
        if weight_td:
            # Look for the inner table that contains the weight values
            weight_table = weight_td.find("table")
            if weight_table:
                first_tr = weight_table.find("tr")
                if first_tr:
                    tds = first_tr.find_all("td")
                    if len(tds) >= 2:
                        weight_imperial = tds[0].get_text(strip=True)
                        weight_metric = tds[1].get_text(strip=True)
    # Combine the weight values into one string.
    weight = f"{weight_imperial} ({weight_metric})" if weight_imperial and weight_metric else weight_imperial

    return category, height, weight

# Suppose df is your DataFrame containing a column "information table" with the HTML string.
# For each row in df, use the helper function to extract the info.
# The zip(*) technique unpacks the tuple returned by extract_info for each row.
df["Pokemon category"], df["Height"], df["Weight"] = zip(*df["information table"].apply(extract_info))

# Check the first few rows to verify the extraction
print(df[["Pokemon category", "Height", "Weight"]].head())


  Pokemon category         Height                 Weight
0     Seed Pokémon  2'04" (0.7 m)     15.2 lbs. (6.9 kg)
1     Seed Pokémon  3'03" (1.0 m)    28.7 lbs. (13.0 kg)
2     Seed Pokémon  6'07" (2.0 m)  220.5 lbs. (100.0 kg)
3   Lizard Pokémon  2'00" (0.6 m)     18.7 lbs. (8.5 kg)
4    Flame Pokémon  3'07" (1.1 m)    41.9 lbs. (19.0 kg)


In [65]:
df = df.drop(['information table'], axis=1)
df

,name,Type,Bio,Pokemon category,Height,Weight
0,Bulbasaur,"Grass, Poison",Bulbasaur using its vines to climb a tree Bulb...,Seed Pokémon,"2'04"" (0.7 m)",15.2 lbs. (6.9 kg)
1,Ivysaur,"Grass, Poison",Ivysaur is a quadrupedal amphibian Pokémon tha...,Seed Pokémon,"3'03"" (1.0 m)",28.7 lbs. (13.0 kg)
2,Venusaur,"Grass, Poison",Venusaur with its bulb closed Venusaur is a sq...,Seed Pokémon,"6'07"" (2.0 m)",220.5 lbs. (100.0 kg)
3,Charmander,Fire,Charmander protecting the flame on its tail fr...,Lizard Pokémon,"2'00"" (0.6 m)",18.7 lbs. (8.5 kg)
4,Charmeleon,Fire,"Charmeleon is a bipedal, reptilian Pokémon . I...",Flame Pokémon,"3'07"" (1.1 m)",41.9 lbs. (19.0 kg)
...,...,...,...,...,...,...
1020,Raging Bolt,"Electric, Dragon",Raging Bolt's mane unfolding Raging Bolt is a ...,Paradox Pokémon,"17'01"" (5.2 m)","1,058.2 lbs. (480.0 kg)"
1021,Iron Boulder,"Rock, Psychic",Iron Boulder's head rotating as it faints Iron...,Paradox Pokémon,"4'11"" (1.5 m)",358.2 lbs. (162.5 kg)
1022,Iron Crown,"Steel, Psychic",Iron Crown's blades extending while using Tach...,Paradox Pokémon,"5'03"" (1.6 m)",343.9 lbs. (156.0 kg)
1023,Terapagos,Normal,Terapagos in its dormant state Normal Form Ter...,Tera Pokémon,"0'08"" (0.2 m)",14.3 lbs. (6.5 kg)


## Output csv file

In [72]:
df.to_csv("pokemon_data.csv", index=False, encoding="utf-8")